<a href="https://colab.research.google.com/github/EvanNT18/BE_Content/blob/main/Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# @title Install Dependency

!pip install fastapi uvicorn yt-dlp SpeechRecognition pydub transformers pyngrok

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!pip install -r requirements.txt




Saving requirements.txt to requirements.txt
User uploaded file "requirements.txt" with length 75 bytes


# Audio


In [16]:
# @title Mengunduh Audio dari Youtube

import yt_dlp

def download_audio_from_youtube(url: str):
    ydl_opts = {
        'format': 'bestaudio/best',  # Pilih format audio terbaik
        'outtmpl': 'downloaded_audio.%(ext)s',  # Output filename
        'postprocessors': [{  # Mengonversi ke format audio yang diinginkan
            'key': 'FFmpegAudioConvertor',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(url, download=True)

    return 'downloaded_audio.mp3'


In [15]:
# @title Mengonversi Audio Menjadi Teks

import speech_recognition as sr

def transcribe_audio_to_text(audio_file: str):
    recognizer = sr.Recognizer()

    # Membuka file audio
    with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)  # Membaca file audio

    try:
        # Mencoba untuk mengenali dan mengonversi audio ke teks menggunakan Google Web Speech API
        text = recognizer.recognize_google(audio)
        return text
    except sr.UnknownValueError:
        return "Google Speech Recognition could not understand the audio"
    except sr.RequestError as e:
        return f"Could not request results from Google Speech Recognition service; {e}"


In [17]:
# @title Summary

from transformers import pipeline

def summarize_text(text: str):
    # Load model peringkasan
    summarizer = pipeline("summarization")

    # Melakukan peringkasan pada teks
    summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
    return summary[0]['summary_text']


# Input Data

In [18]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pyngrok import ngrok

# Membuat instance FastAPI
app = FastAPI()

@app.post("/summarize/")
async def summarize_youtube_video(url: str):
    try:
        # Mengunduh audio dari URL YouTube
        audio_file = download_audio_from_youtube(url)

        # Menyalin audio ke teks
        text = transcribe_audio_to_text(audio_file)

        if text.startswith("Error"):
            raise HTTPException(status_code=400, detail="Error transcribing audio")

        # Menyimpulkan teks
        summary = summarize_text(text)

        return JSONResponse(content={"summary": summary})

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


public_url = ngrok.connect(8000)
print(f"FastAPI running on: {public_url}")


ERROR:pyngrok.process.ngrok:t=2025-04-03T18:17:16+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-03T18:17:16+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.